1. A description of the problem and a discussion of the background:

Scenerio:

On March 17th, 2020, former New England Patriot QB Tom Brady decided to agree to a deal with the Tampa Bay Buccaneers. As a lifelong Bucs fan, this is the greatest moment in my fandom since their Super Bowl 37 victory over the Oakland Raiders. It is time to make the move to Tampa Bay to capitalize on the next 3-year dynasty in the NFL. My goal is to open a new sports bar called "Tampa Tommy’s." This is where all the die-hard Bucs fans will come to cheer on the home team in pursuit of another Super Bowl this century.

Problem:

I have not lived in Tampa, Florida since I was a 1 year old, and I have no clue where all the existing establishments are. My goal is to have a sports bar close to Raymond James Stadium so that when fans cannot afford tickets, they can still tailgate at the stadium and then come over to Tampa Tommy’s for the actual game.

Goal:

To find the perfect location for Tampa Tommy’s so that it is a thriving business before the team becomes mediocre again.

2. A description of the data and how it will be used to solve the problem

The data we will be using is listed below. 

We want to find the venues around Raymond James Stadium, cluster them, and determine which block is the best place to try and startup a new restaurant.

Foursquare add geopy data will be used to look up these venues.

We will then cluster the data we collect to determine which area would be the best place to start Tampa Tommy’s

Once the cluster is determined, we will look up rental properties in the area to determine where Tampa Tommy's will be located permanently.

In [2]:
import numpy as np
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
!pip install folium
import folium
print('Libraries Loaded')

     |████████████████████████████████| 92kB 9.2MB/s  eta 0:00:01
Libraries Loaded


In [77]:
#One Buc Place Location in Tampa, Florida
address='W Dr Martin Luther King Jr Blvd, Tampa'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


27.9813767 -82.4948689


In [60]:
#Latitude and Longitude of One Buc Place
obp_lat=27.9813767
obp_long=-82.4948689

In [78]:
#My Foursquare Info
CLIENT_ID='SB5****************************************'
CLIENT_SECRET='5UI************************************'
VERSION = '20200101'

In [79]:
#Generate a url for the foursquare api
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, obp_lat, obp_long)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=SB55ZZTADJ1VERRXNH45HX3ZJWS5PICL5AMGLUKPJIN1RRMI&client_secret=5UI1DU1GRK5LGCAI3KD4O53QGKBTUK51L5IHNL1KZZUBDTVL&v=20200101&ll=27.9813767,-82.4948689'

In [80]:
#turn results into json format
results=requests.get(url).json()

In [81]:
#rows
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [82]:
#colums
venue_list=results['response']['groups'][0]['items']
    
obp_venues=json_normalize(venues)

filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

obp_venues=obp_venues.loc[:, filtered_columns]

obp_venues['venue.categories']=obp_venues.apply(get_category_type, axis=1)

obp_venues.columns=[col.split(".")[-1] for col in obp_venues.columns]

#table of venues
obp_venues

,name,categories,lat,lng
0,Happy Fish Seafood Restaurant,Spanish Restaurant,27.975567,-82.486323
1,Raymond James Stadium,Football Stadium,27.975832,-82.503355
2,Starbucks,Coffee Shop,27.981005,-82.488442
3,Al Lopez Park,Park,27.989207,-82.503020
4,Raymond James Stadium Luxury Suite,Football Stadium,27.976280,-82.503938
5,Panera Bread,Bakery,27.980857,-82.499157
6,West Tampa Sandwich Shop,Cuban Restaurant,27.973415,-82.485005
7,Montauro Restaurante,Italian Restaurant,27.974365,-82.486587
8,George M Steinbrenner Field,Baseball Stadium,27.979863,-82.507246
9,Bucs Pirate Ship,Football Stadium,27.974618,-82.503288


In [83]:
#Map of venues near One Buc Place (used Red and Pewter for the festive occasion)

tampa_map=folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, label in zip(obp_venues['lat'], obp_venues['lng'], obp_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker([lat, lng], radius=15, color='red', fill_color='#696969', fill_opacity=0.5,).add_to(tampa_map)  
    
tampa_map